# SHOPEE  Data Analytics Conmpetition - Multi-Channel Contacts Problems

![shopee](./shopeee.png)

## > 背景 （翻譯於原始文件）

客戶服務是Shopee所提供的業務中重要的部分，為客戶提供優質服務對於業務增長和品牌形象至關重要。而Shopee的目標是在最短的時間以及耗費最少客戶工作量去解決客戶的問題。客戶工作量的一種衡量標準是客戶在特定問題上必須接近客戶服務的次數，這也稱為度量“重複聯繫率Repeat Contact Rate"或RCR。 

Shopee希望藉由研究RCR以提高對客戶服務效率，使客戶可以通過各種渠道（例如即時聊天功能，填寫某些表格或致電尋求幫助）與客服聯繫。每次客戶使用新的聯繫方式與Shopee聯繫時，都會自動生成新的資料。當同一位客戶使用不同的電話號碼或電子郵件地址與Shopee聯繫，從而導致同一問題的多筆資料時，就會出現問題。因此，我們面臨的挑戰是確定如何將相關資料合併在一起以創建客戶問題的完整輪廓並最終確定RCR。

##  > 競賽內容

針對每筆數據，去確認每筆要求數據的聯繫人資訊，如果他們具有相同的聯繫信息即代表此要求來自相同的消費者。

（假定來自同一電話號碼/電子郵件的所有聯繫人都是同一用戶。） 

## > 案例條件

在這些資料當中，只要他們擁有相同的資訊（Email, Phone, OrderID）即代表他們還自同一個用戶。

1. A 與 B 擁有相同的OrderID
2. B 與 C 擁有相同的Phone
3. C 與 D 擁有相同的Email
4. 此時 A <-> B <-> C <-> D 這四個要求都被視為有關係，所以這四個要求均代表同一個用戶。

## > 實作策略
在本次競賽當中，我打算分為兩個階段來進行：
* 第一階段 直接關係：先把1~3條件有直接關係的id找出來
* 第二階段 間接關係：進一步去尋找如第四個條件那種間接關係的關聯id
* 最後階段：計算總和Contacts數以及Submission格式的調整

## 讀取檔案

此次競賽的資料集是以json的方式，一樣以pandas做讀取，輕鬆寫意。

而此資料集總共有500,000筆資料，並且有涵蓋ID, Email, Phone, OrderID, Contacts 等五個特徵，


| Feature | Describe |
| :---: | :---: |
| ID | 與index相同，僅代表數據的流水號 |
| Email, Phone, OrderID | 此三個特徵為本數據集的重點特徵，分別代表這次要求申請人的郵件信箱、電話號碼、以及要求序號 |
| Contacts | 此特徵表示該要求的聯繫次數，在最後的答案提交需要去統整每個聯絡人的聯繫次數，即可進一步去分析其RCR |

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
df = pd.read_json("/Users/charliesun/Desktop/Data Science Folder/SHOPEE CODE LEAGUE/contacts.json",encoding="utf-8", orient='records')
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,
...,...,...,...,...,...
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


## 直接關係
首先，我想要從最基本且好觀察的關係下手，也就是當Email, Phone, OrderId相同的時候所建立的第一層關係開始去做。<br>
而我打算是三個特徵分開來去做，但是因為其手法相同，所以完成一次就等於將三個特徵分好類。<br><br/>
以下以Email作為範例：<br>
首先我將資料切成較小的子集來做，<br>
且由於json的檔案被讀取之後，其中的空值會以空字串的方式呈現，為了避免被程式判定空字串為相同的元素，先將其轉為nan。

In [2]:
email_part = df[['Id','Email']]
email_part['Email'] = email_part['Email'].replace('', np.nan)
email_part.dropna(axis = 0, subset = ['Email'], inplace = True)
email_part.reset_index(inplace = True, drop = True)

接下來是比較tricky的部分，這邊將思考模式顛倒過來，不是以id的角度出發，而是以Email的角度作為出發點。

這邊做了一個Dictionary，<br>
其key值為在原始資料集中出現過不重複的Email<br>
而Value值為擁有這個Email所對應的id。 （可同理推廣至其他特徵）

這樣就可以很快知道哪些ID擁有一樣的資訊，
並且將此字典轉成DataFrame的型態。

In [3]:
email_id_dict = {k: list(v) for k, v in email_part.groupby('Email')['Id']}
email_df = pd.DataFrame()
email_df['Email'] = email_id_dict.keys()
email_df['email_related_id'] = email_id_dict.values()
email_df.head(5)

,Email,email_related_id
0,AAAQbNfGbLAeOvw@hotmail.com,"[112387, 470075]"
1,AAAgyKTteOjasN@qq.com,[338154]
2,AABZmKeEldvO@gmail.com,[426495]
3,AABneiGcqcz@qq.com,[37868]
4,AABtOIZmdYdKGnidAK@yahoo.com,[137956]


接著再將我們思考的角度轉回來，回到原本的資料型態以Id為出發的角度，<br>
在這裡使用到Left Join的概念，把我們剛剛做出來的Email資訊，<br>
以Email資訊作為連結點join到原本的資料集上。

In [4]:
ex_df = pd.merge(df, email_df, how = 'left', on = 'Email')
ex_df.head(5)

,Id,Email,Phone,Contacts,OrderId,email_related_id
0,0,gkzAbIy@qq.com,,1,,[0]
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,NaN
2,2,,9125983679,0,,NaN
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,[3]
4,4,,300364407,2,,NaN


現在 email_related_id 這個特徵即表示所有與這筆資料擁有相同Email的ID，

而對於第一層個關係皆可以以這種方式呈現，所以利用函式來簡化操作。

In [5]:
featuretouse = ['Email','Phone','OrderId']

def match_the_feature(df, x):
    part = df[['Id',x]]
    part[x] = part[x].replace('', np.nan) # 從json讀取的檔案當中，缺失值會是空字串的形式，所以先轉成nan
    part.dropna(axis = 0, subset = [x], inplace = True)
    part.reset_index(inplace = True, drop = True)

    id_dict = {k: list(v) for k, v in part.groupby(x)['Id']}
    
    part_df = pd.DataFrame()
    part_df[x] = id_dict.keys()
    part_df['{}_related_id'.format(x).lower()] = id_dict.values()
    
    df = pd.merge(df, part_df, how = 'left', on = x)
    return df

# Match the Email, Phone, OrderId
for i in featuretouse:
    df = match_the_feature(df, i)
    
df.head(5)

,Id,Email,Phone,Contacts,OrderId,email_related_id,phone_related_id,orderid_related_id
0,0,gkzAbIy@qq.com,,1,,[0],NaN,NaN
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,NaN,[1],"[1, 2458, 476346]"
2,2,,9125983679,0,,NaN,"[2, 348955]",NaN
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,[3],NaN,[3]
4,4,,300364407,2,,NaN,[4],NaN


而當我們有email, phone, orderid_related_id之後，先針對NaN值進行處理，將他轉為空的list <br>
這樣就可以針對這三個特徵進行總和，且最後利用一個小技巧：
* 當x為一個list時，list (set (x))可以去除list中的重複值，相當於np.unique()的功能

>而此時的total_related_id已經可以表達出第一層的結果，<br>
>這邊以id = 6的那筆數據為例：<br>
>[6, 38, 32871, 142067] 指這四個已經被認定為是有關係的id，因為他們可能有email、phone或者orderid其中之一相同。

In [6]:
# COMBINE & TRANSFER
df['phone_related_id'] = df['phone_related_id'].fillna("").apply(list)
df['email_related_id'] = df['email_related_id'].fillna("").apply(list)
df['orderid_related_id'] = df['orderid_related_id'].fillna("").apply(list)

df['total_related_id'] = df['phone_related_id'] + df['email_related_id'] + df['orderid_related_id']
df['total_related_id'] = df['total_related_id'].apply(lambda x: list(set(x)))
df.head(5)

,Id,Email,Phone,Contacts,OrderId,email_related_id,phone_related_id,orderid_related_id,total_related_id
0,0,gkzAbIy@qq.com,,1,,[0],[],[],[0]
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,[],[1],"[1, 2458, 476346]","[476346, 1, 2458]"
2,2,,9125983679,0,,[],"[2, 348955]",[],"[2, 348955]"
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,[3],[],[3],[3]
4,4,,300364407,2,,[],[4],[],[4]


## 間接關係

目前已經完成了最明顯的直接關係，但依據競賽的要求，還有一層間接的關係需要去處理。<br><br/>
這邊以下面的這組數據表格作為範例：<br>
這邊顯示與Id=2有相關的序號僅有348955，原因是因為在Id=2的這筆資料內email, orderid都是缺失值，所以找不到與其他的關聯。<br>
但 Id=348955 的這筆資料的email跟orderid就與159312, 322639相同，<br>
所以id = 2的應該與id = 159312, 322639 的數據有間接的關係。 <br>

所以在正確答案之中，這四個id的total_related_id應該都要是一樣為 [2, 159312 ,348955 ,322639]




In [7]:
df[df['Id'].isin([2,348955,159312,322639])].iloc[:,-4:]

,email_related_id,phone_related_id,orderid_related_id,total_related_id
2,[],"[2, 348955]",[],"[2, 348955]"
159312,"[159312, 322639, 348955]",[159312],[159312],"[159312, 348955, 322639]"
322639,"[159312, 322639, 348955]",[],[322639],"[159312, 348955, 322639]"
348955,"[159312, 322639, 348955]","[2, 348955]",[],"[159312, 2, 348955, 322639]"


所以這邊針對total_related_Id這個欄位去做調整，<br>
我依照順序去讀取每一個list裡面的數字，並且同時去讀取相對id的list將他做合併，最後再去除掉重複值即可。<br>

ex:<br>
id = 2 -->  [2,348955] (2的List) + [159312, 2, 348955, 322639] (348955的List) <br>
id = 159312 --> [159312,348955,322639] (159312的List) + [159312,348955,322639] (322639的List) + [2,159312,348955,322639] (348955的List)

而因為每筆資料的順序不一定，所以我讓整個比對的流程跑了三次，確保每一筆的結果會是一樣的。

In [8]:
## COMPARE THE EACH ID RESULT
def sum_the_id(x):
    tt = []
    for i in x:
        tt += df['total_related_id'][i]
    tt = list(set(tt))
    return tt

# to match the whole ID for 5 times
for _ in range(3):
    df['total_related_id'] = df['total_related_id'].apply(lambda x: sum_the_id(x))
    
df['total_related_id'].apply(lambda x: x.sort())
df[df['Id'].isin([2,348955,159312,322639])].iloc[:,-4:]

,email_related_id,phone_related_id,orderid_related_id,total_related_id
2,[],"[2, 348955]",[],"[2, 159312, 322639, 348955]"
159312,"[159312, 322639, 348955]",[159312],[159312],"[2, 159312, 322639, 348955]"
322639,"[159312, 322639, 348955]",[],[322639],"[2, 159312, 322639, 348955]"
348955,"[159312, 322639, 348955]","[2, 348955]",[],"[2, 159312, 322639, 348955]"


## 計算Contacts以及轉換至Submission格式

最後一步僅將每個Id所對應的contacts去做連結以及加總， <br>
最後再將Dataframe調整成Submission所要求的字串格式即完成了這份題目～

In [9]:
## DICT OF THE CONTACTS
contacts = df['Contacts'].to_dict()

def contacts_sum(x):
    a=0
    for i in x:
        a += contacts[i]
        
    return a

# Count_the_contacts
df['Contact_total'] = df['total_related_id'].apply(lambda x: contacts_sum(x))


#transfer to the submission format
df['total_related_id_2'] = df['total_related_id'].apply(lambda x: '-'.join(str(i) for i in x))
df['Contact_total'] = df['Contact_total'].apply(lambda x: str(x))
df['final_answer'] = df['total_related_id_2'] + ', ' + df['Contact_total']

#SUBMISSION

submission = pd.DataFrame()
submission['ticket_id'] = df['Id']
submission['ticket_trace/contact'] = df['final_answer']
submission.head(10)

,ticket_id,ticket_trace/contact
0,0,"0, 1"
1,1,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,"2-159312-322639-348955, 4"
3,3,"3, 0"
4,4,"4, 2"
5,5,5-50-212533-215197-226720-383605-404324-458692...
6,6,"6-38-32871-142067-236367, 13"
7,7,"7, 1"
8,8,"8-183160-406623, 5"
9,9,"9-13-16708-33415-343161-417916-468927-484896, 8"


## 完整程式碼

In [10]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
df = pd.read_json("/Users/charliesun/Desktop/Data Science Folder/SHOPEE CODE LEAGUE/contacts.json",encoding="utf-8", orient='records')

featuretouse = ['Email','Phone','OrderId']

def match_the_feature(df, x):
    part = df[['Id',x]]
    part[x] = part[x].replace('', np.nan) # 從json讀取的檔案當中，缺失值會是空字串的形式，所以先轉成nan
    part.dropna(axis = 0, subset = [x], inplace = True)
    part.reset_index(inplace = True, drop = True)

    id_dict = {k: list(v) for k, v in part.groupby(x)['Id']}
    
    part_df = pd.DataFrame()
    part_df[x] = id_dict.keys()
    part_df['{}_related_id'.format(x).lower()] = id_dict.values()
    
    df = pd.merge(df, part_df, how = 'left', on = x)
    return df

# Match the Email, Phone, OrderId
for i in featuretouse:
    df = match_the_feature(df, i)
    
# COMBINE & TRANSFER

df['phone_related_id'] = df['phone_related_id'].fillna("").apply(list)
df['email_related_id'] = df['email_related_id'].fillna("").apply(list)
df['orderid_related_id'] = df['orderid_related_id'].fillna("").apply(list)

df['total_related_id'] = df['phone_related_id'] + df['email_related_id'] + df['orderid_related_id']
df['total_related_id'] = df['total_related_id'].apply(lambda x: list(set(x)))

## COMPARE THE EACH ID RESULT
def sum_the_id(x):
    tt = []
    for i in x:
        tt += df['total_related_id'][i]
    tt = list(set(tt))
    return tt

# to match the whole ID for 3 times
for _ in range(3):
    df['total_related_id'] = df['total_related_id'].apply(lambda x: sum_the_id(x))
    
df['total_related_id'].apply(lambda x: x.sort())


## DICT OF THE CONTACTS
contacts = df['Contacts'].to_dict()

def contacts_sum(x):
    a=0
    for i in x:
        a += contacts[i]
        
    return a
# Count_the_contacts
df['Contact_total'] = df['total_related_id'].apply(lambda x: contacts_sum(x))


#transfer to the submission format
df['total_related_id_2'] = df['total_related_id'].apply(lambda x: '-'.join(str(i) for i in x))
df['Contact_total'] = df['Contact_total'].apply(lambda x: str(x))
df['final_answer'] = df['total_related_id_2'] + ', ' + df['Contact_total']

#SUBMISSION

submission = pd.DataFrame()
submission['ticket_id'] = df['Id']
submission['ticket_trace/contact'] = df['final_answer']
# submission.to_csv('Submission.csv', index = False)